<a href="https://colab.research.google.com/github/ThatiFlowers/Treinamento/blob/main/CRUD_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

def create_tables():
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        preco REAL NOT NULL,
        categoria TEXT NOT NULL
    )
    """)

    conexao.commit()
    conexao.close()

create_tables()


In [54]:
from os import name
# Criando a classe de produtos/ repositório(domínio)
class produtos:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria

In [55]:
# Create the 'repository' directory if it doesn't exist
!mkdir -p repository

In [56]:
%%writefile repository/produto_repository.py
# Define the produtorepository class here
class produtorepository:
    def __init__(self):
        pass # You will need to add your repository logic here

Overwriting repository/produto_repository.py


In [57]:
%%writefile domain/produto.py
# Criando a classe de produtos
class produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria

Overwriting domain/produto.py


In [58]:
def cadastrar_produtos(nome, preco, categoria):
  conexao = conexao_bd()
  cursor=conexao.cursor()
  curso.execute("Insert into produtos(nome, preco,categoria)values (?,?,?)",(produto.nome, profuto.preco, produto.categoria))
  conexao.commit()
  conexao.close()

In [59]:
def listar_produtos():
  conexao = conexao_bd()
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  rows = cursor.fetchall()
  conexao.close()
  return rows

  produtos =[]
  for row in rows:
    produto = produtos(row[1], row[2], row[3])
    produtos.append(produto)
    return produtos

    def produtos_id(id):
      conexao = conexao_bd()
      cursor = conexao . cursor()
      cursor.execute("select from produtos where id =?", (id,))
      row = cursor.fetchone()
      conexao.close()
      produto=produtos(row[1], row[2], row [3])
      return produto

